In [1]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

In [2]:
import h1st as h1
h1.init()

In [3]:
from h1st.tuner import HyperParameterTuner

In [4]:
from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier

In [5]:
tuner = HyperParameterTuner()

In [6]:
# 'tuning_param': {'lr': float, 'units': int}
# 'target_metrics': {'accuracy': 'minimize'}

# parameters = {
#     'lr': {'min': 0.001, 'max': 0.1},
#     'n_layer': {'min': 1, 'max': 8},    
#     'units': [4, 8, 16, 32, 64]
# }

parameters = [
    {
        'name': 'lr',
        'type': 'float',
        'min': 0.001,
        'max': 0.1,
        'choice': []
    },
    {
        'name': 'n_layer',
        'type': 'int',
        'min': 1,
        'max': 8,
        'choice': []
    },
    {
        'name': 'units',
        'type': 'int',
        'min': 4,
        'max': 32,
        'choice': []
    },    
#     {
#         'name': 'units',
#         'type': int,
#         'min': None,
#         'max': None,
#         'choice': [4, 8, 16, 32, 64]
#     }    
]
target_metrics = 'accuracy'
options = {
    'num_samples': 10,
    'stopping_criteria': {
        'training_iteration': 5,
    }
}

If it is a choice, then we need to use other model 

In [7]:
analysis = tuner.run(TensorflowMLPClassifier, parameters, target_metrics, options)

Trial name,status,loc,lr,n_layer,units,acc,iter,total time (s)
H1stModelTrainable_7c8b9c18,TERMINATED,,0.0380795,7.655,24.4958,0.816061,4,8.54872
H1stModelTrainable_7c8bd9da,TERMINATED,,0.0602672,2.09213,8.36785,0.820707,5,6.56767
H1stModelTrainable_7c8c05b8,TERMINATED,,0.00675028,7.06323,20.8312,0.817576,1,2.71082
H1stModelTrainable_7c8c3772,TERMINATED,,0.0710992,1.14409,31.1575,0.812929,5,5.70344
H1stModelTrainable_7c8c688c,TERMINATED,,0.0834118,2.48637,9.0911,0.816566,5,6.83687
H1stModelTrainable_7c8c9334,TERMINATED,,0.019157,3.1297,18.6932,0.809697,1,1.71954
H1stModelTrainable_7c8cbd00,TERMINATED,,0.0437626,3.0386,21.1319,0.816566,1,1.7025
H1stModelTrainable_7c8ceb2c,TERMINATED,,0.0148099,3.04501,14.2581,0.815859,4,6.2019
H1stModelTrainable_7c8d12e6,TERMINATED,,0.0461509,6.49623,9.59087,0.816061,4,7.4614
H1stModelTrainable_7c8d3ce4,TERMINATED,,0.0519092,5.1469,5.30061,0.819293,5,8.30787


took   20.56 seconds (   0.34 minutes)
